![OpenSARlab notebook banner](NotebookAddons/blackboard-banner.png)

# Prepare a SAR Data Stack
### Joseph H Kennedy and Alex Lewandowski; Alaska Satellite Facility

This notebook downloads an ASF-HyP3 RTC project and prepares a deep multi-temporal SAR image data stack for use in other notebooks.

<img style="padding: 7px" src="NotebookAddons/UAFLogo_A_647.png" width="170" align="right"/></font>

### Important Note about JupyterHub

**Your JupyterHub server will automatically shutdown when left idle for more than 1 hour. Your notebooks will not be lost but you will have to restart their kernels and re-run them from the beginning. You will not be able to seamlessly continue running a partially run notebook.**

In [1]:
import url_widget as url_w
notebookUrl = url_w.URLWidget()
display(notebookUrl)


URLWidget()

In [2]:
from IPython.display import Markdown
from IPython.display import display

notebookUrl = notebookUrl.value
user = !echo $JUPYTERHUB_USER
env = !echo $CONDA_PREFIX
if env[0] == '':
    env[0] = 'Python 3 (base)'
if env[0] != '/home/jovyan/.local/envs/rtc_analysis':
    display(Markdown(f'<text style=color:red><strong>WARNING:</strong></text>'))
    display(Markdown(f'<text style=color:red>This notebook should be run using the "rtc_analysis" conda environment.</text>'))
    display(Markdown(f'<text style=color:red>It is currently using the "{env[0].split("/")[-1]}" environment.</text>'))
    display(Markdown(f'<text style=color:red>Select the "rtc_analysis" from the "Change Kernel" submenu of the "Kernel" menu.</text>'))
    display(Markdown(f'<text style=color:red>If the "rtc_analysis" environment is not present, use <a href="{notebookUrl.split("/user")[0]}/user/{user[0]}/notebooks/conda_environments/Create_OSL_Conda_Environments.ipynb"> Create_OSL_Conda_Environments.ipynb </a> to create it.</text>'))
    display(Markdown(f'<text style=color:red>Note that you must restart your server after creating a new environment before it is usable by notebooks.</text>'))

<text style=color:red><strong>WARNING:</strong></text>

<text style=color:red>This notebook should be run using the "rtc_analysis" conda environment.</text>

<text style=color:red>It is currently using the "hydrosar_rtc_era5" environment.</text>

<text style=color:red>Select the "rtc_analysis" from the "Change Kernel" submenu of the "Kernel" menu.</text>

<text style=color:red>If the "rtc_analysis" environment is not present, use <a href="https://opensciencelab.asf.alaska.edu/lab/smce-prod-opensarlab/user/jknicely/notebooks/conda_environments/Create_OSL_Conda_Environments.ipynb"> Create_OSL_Conda_Environments.ipynb </a> to create it.</text>

<text style=color:red>Note that you must restart your server after creating a new environment before it is usable by notebooks.</text>

## 0. Importing Relevant Python Packages

In this notebook we will use the following scientific libraries:

1. [GDAL](https://www.gdal.org/) is a software library for reading and writing raster and vector geospatial data formats. It includes a collection of programs tailored for geospatial data processing. Most modern GIS systems (such as ArcGIS or QGIS) use GDAL in the background.
1. [NumPy](http://www.numpy.org/) is one of the principal packages for scientific applications of Python. It is intended for processing large multidimensional arrays and matrices, and an extensive collection of high-level mathematical functions and implemented methods makes it possible to perform various operations with these objects.

**Our first step is to import them:**

In [8]:
%%capture
import copy
from datetime import datetime, timedelta, timezone
import json # for loads
from pathlib import Path
import re
import shutil
import warnings
import rasterio
from rasterio.merge import merge

from osgeo import gdal
import numpy as np

from IPython.display import display, clear_output, Markdown

import opensarlab_lib as asfn # this gets used ~22 times

from hyp3_sdk import Batch, HyP3

## 1. Load Your Own Data Stack Into the Notebook

This notebook assumes that you've created your own data stack over your personal area of interest using the [Alaska Satellite Facility's](https://www.asf.alaska.edu/) value-added product system HyP3, available via [ASF Data Search (Vertex)](https://search.asf.alaska.edu/#/). HyP3 is an ASF service used to prototype value added products and provide them to users to collect feedback.

We will retrieve HyP3 data via the hyp3_sdk. As both HyP3 and the Notebook environment sit in the [Amazon Web Services (AWS)](https://aws.amazon.com/) cloud, data transfer is quick and cost effective.

---

Before we download anything, create a working directory for this analysis.

**Select or create a working directory for the analysis:**

In [4]:
while True:
    print(f"Current working directory: {Path.cwd()}")
    data_dir = Path(Path.cwd()/'Dataset'/input(f"\nPlease enter the name of a directory in which to store your data for this analysis."))
    if data_dir == Path('.'):
        continue
    if data_dir.is_dir():
        contents = data_dir.glob('*')
        if len(list(contents)) > 0:
            choice = asfn.handle_old_data(data_dir)
            if choice == 1:
                if data_dir.exists():
                    shutil.rmtree(data_dir)
                data_dir.mkdir(parents=True, exist_ok=True)
                break
            elif choice == 2:
                break
            else:
                clear_output()
                continue
        else:
            break
    else:
        data_dir.mkdir(parents=True, exist_ok=True)
        break

Current working directory: /home/jovyan/hydrosar



Please enter the name of a directory in which to store your data for this analysis. Nepal_sF


**Define absolute path to  analysis directory:**

In [5]:
analysis_directory = Path.cwd().joinpath(data_dir)
print(f"analysis_directory: {analysis_directory}")

analysis_directory: /home/jovyan/hydrosar/Dataset/Nepal_sF


**Create a HyP3 object and authenticate:**

In [6]:
hyp3 = HyP3(prompt=True)

NASA Earthdata Login username:  jknicely_alaska.edu
NASA Earthdata Login password:  ········


**Select a product type to download:**

In [9]:
job_types = ['RTC_GAMMA', 'INSAR_GAMMA', 'AUTORIFT']
job_type = asfn.select_parameter(job_types)
job_type

RadioButtons(layout=Layout(min_width='800px'), options=('RTC_GAMMA', 'INSAR_GAMMA', 'AUTORIFT'), value='RTC_GA…

**Decide whether to search for a HyP3 project or jobs unattached to a project:** 

In [10]:
options = ['project', 'projectless jobs']
search_type = asfn.select_parameter(options, '')
print("Select whether to search for HyP3 Project or HyP3 Jobs unattached to a project")
display(search_type)

Select whether to search for HyP3 Project or HyP3 Jobs unattached to a project


RadioButtons(layout=Layout(min_width='800px'), options=('project', 'projectless jobs'), value='project')

**List projects containing active products of the type chosen in the previous cell and select one:**

In [11]:
my_hyp3_info = hyp3.my_info()
active_projects = dict()

if search_type.value == 'project':
    for project in my_hyp3_info['job_names']:
        batch = Batch()
        batch = hyp3.find_jobs(name=project, job_type=job_type.value).filter_jobs(running=False, include_expired=False)
        if len(batch) > 0:
            active_projects.update({batch.jobs[0].name: batch})

    if len(active_projects) > 0:
        display(Markdown("<text style='color:darkred;'>Note: After selecting a project, you must select the next cell before hitting the 'Run' button or typing Shift/Enter.</text>"))
        display(Markdown("<text style='color:darkred;'>Otherwise, you will rerun this code cell.</text>"))
        print('\nSelect a Project:')
        project_select = asfn.select_parameter(active_projects.keys())
        display(project_select)
if search_type.value == 'projectless jobs' or len(active_projects) == 0:
    project_select = False
    if search_type.value == 'project':
        print(f"There were no {job_type.value} jobs found in any current projects.\n")
    jobs = hyp3.find_jobs(job_type=job_type.value).filter_jobs(running=False, include_expired=False)
    orphaned_jobs = Batch()
    for j in jobs:
        if not j.name:
            orphaned_jobs += j
    jobs = orphaned_jobs

    if len(jobs) > 0:
        print(f"Found {len(jobs)} {job_type.value} jobs that are not part of a project.")
        print(f"Select the jobs you wish to download")
        jobs = {i.files[0]['filename']: i for i in jobs}
        jobs_select = asfn.select_mult_parameters(jobs.keys(), '', width='500px')
        display(jobs_select)
    else:
        print(f"There were no {job_type.value} jobs found that are not part of a project either.")
    

<text style='color:darkred;'>Note: After selecting a project, you must select the next cell before hitting the 'Run' button or typing Shift/Enter.</text>

<text style='color:darkred;'>Otherwise, you will rerun this code cell.</text>


Select a Project:


RadioButtons(layout=Layout(min_width='800px'), options=('Nepal',), value='Nepal')


**Select a date range of products to download:**

In [12]:
if project_select:
    batch = active_projects[project_select.value]
else:
    batch = Batch()
    for j in jobs_select.value:
        batch += jobs[j]

display(Markdown("<text style='color:darkred;'>Note: After selecting a date range, you should select the next cell before hitting the 'Run' button or typing Shift/Enter.</text>"))
display(Markdown("<text style='color:darkred;'>Otherwise, you may simply rerun this code cell.</text>"))
print('\nSelect a Date Range:')
dates = asfn.get_job_dates(batch)
date_picker = asfn.gui_date_picker(dates)
date_picker

<text style='color:darkred;'>Note: After selecting a date range, you should select the next cell before hitting the 'Run' button or typing Shift/Enter.</text>

<text style='color:darkred;'>Otherwise, you may simply rerun this code cell.</text>


Select a Date Range:


SelectionRangeSlider(description='Dates', index=(0, 1488), layout=Layout(width='500px'), options=((' 03/23/202…

**Save the selected date range and remove products falling outside of it:**

In [13]:
date_range = asfn.get_slider_vals(date_picker)
date_range[0] = date_range[0].date()
date_range[1] = date_range[1].date()
print(f"Date Range: {str(date_range[0])} to {str(date_range[1])}")
batch = asfn.filter_jobs_by_date(batch, date_range)

Date Range: 2020-03-23 to 2024-04-19


**Gather the available paths and orbit directions for the remaining products:**

In [14]:
batch

Batch([Job.from_dict({'job_id': '9e40e530-b807-49d9-89c5-2d3ecba4cbf1', 'job_type': 'RTC_GAMMA', 'request_time': '2024-04-24T22:52:01+00:00', 'status_code': 'SUCCEEDED', 'user_id': 'jknicely_alaska.edu', 'name': 'Nepal', 'job_parameters': {'dem_matching': False, 'dem_name': 'copernicus', 'granules': ['S1A_IW_SLC__1SDV_20200323T123842_20200323T123909_031803_03AB54_868E'], 'include_dem': True, 'include_inc_map': False, 'include_rgb': False, 'include_scattering_area': False, 'radiometry': 'gamma0', 'resolution': 30, 'scale': 'power', 'speckle_filter': False}, 'files': [{'filename': 'S1A_IW_20200323T123842_DVP_RTC30_G_gpuned_96AF.zip', 's3': {'bucket': 'hyp3-edc-prod-contentbucket-1fv14ed36ifj6', 'key': '9e40e530-b807-49d9-89c5-2d3ecba4cbf1/S1A_IW_20200323T123842_DVP_RTC30_G_gpuned_96AF.zip'}, 'size': 546699357, 'url': 'https://d3gm2hf49xd6jj.cloudfront.net/9e40e530-b807-49d9-89c5-2d3ecba4cbf1/S1A_IW_20200323T123842_DVP_RTC30_G_gpuned_96AF.zip'}], 'logs': [], 'browse_images': ['https://d3g

In [15]:
display(Markdown("<text style='color:darkred;'><text style='font-size:150%;'>This may take some time for projects containing many jobs...</text></text>"))
asfn.set_paths_orbits(batch)
paths = set()
orbit_directions = set()
for p in batch:
    paths.add(p.path)
    orbit_directions.add(p.orbit_direction)
paths.add('All Paths')
display(Markdown(f"<text style=color:blue><text style='font-size:175%;'>Done.</text></text>"))

<text style='color:darkred;'><text style='font-size:150%;'>This may take some time for projects containing many jobs...</text></text>

/home/jovyan/.local/envs/hydrosar_rtc_era5/lib/python3.12/site-packages/tzlocal/unix.py:193: UserWarning: Can not find any timezone configuration, defaulting to UTC.
  warnings.warn("Can not find any timezone configuration, defaulting to UTC.")


<text style=color:blue><text style='font-size:175%;'>Done.</text></text>

---
**Select a path or paths (use shift or ctrl to select multiple paths):**

In [16]:
display(Markdown("<text style='color:darkred;'>Note: After selecting a path, you must select the next cell before hitting the 'Run' button or typing Shift/Enter.</text>"))
display(Markdown("<text style='color:darkred;'>Otherwise, you will simply rerun this code cell.</text>"))
print('\nSelect a Path:')
path_choice = asfn.select_mult_parameters(paths)
path_choice

<text style='color:darkred;'>Note: After selecting a path, you must select the next cell before hitting the 'Run' button or typing Shift/Enter.</text>

<text style='color:darkred;'>Otherwise, you will simply rerun this code cell.</text>


Select a Path:


SelectMultiple(layout=Layout(height='38px', width='175px'), options=(56, 'All Paths'), value=())

**Save the selected flight path/s:**

In [17]:
flight_path = path_choice.value
if flight_path:
    if flight_path:
        print(f"Flight Path: {flight_path}")
    else:
        print('Flight Path: All Paths')
else:
    print("WARNING: You must select a flight path in the previous cell, then rerun this cell.")

Flight Path: (56,)


**Select an orbit direction:**

In [18]:
if len(orbit_directions) > 1:
    display(Markdown("<text style='color:red;'>Note: After selecting a flight direction, you must select the next cell before hitting the 'Run' button or typing Shift/Enter.</text>"))
    display(Markdown("<text style='color:red;'>Otherwise, you will simply rerun this code cell.</text>"))
print('\nSelect a Flight Direction:')
direction_choice = asfn.select_parameter(orbit_directions, 'Direction:')
direction_choice


Select a Flight Direction:


RadioButtons(description='Direction:', layout=Layout(min_width='800px'), options=('ASCENDING',), value='ASCEND…

**Save the selected orbit direction:**

In [19]:
direction = direction_choice.value
print(f"Orbit Direction: {direction}")

Orbit Direction: ASCENDING


**Filter jobs by path and orbit direction:**

In [20]:
batch = asfn.filter_jobs_by_path(batch, flight_path)
batch = asfn.filter_jobs_by_orbit(batch, direction)
print(f"There are {len(batch)} products to download.")

There are 123 products to download.


**Download the products, unzip them into a directory named after the product type, and delete the zip files:**

Note: The below code cell will return an error message if all VERTEX jobs within the selected timeframe have not been completed at the time of download. 

In [ ]:
products_path = analysis_directory.joinpath(job_type.value)
print(products_path)
if not products_path.is_dir():
    products_path.mkdir()

print(f"\nProject: {batch.jobs[0].name}")
project_zips = batch.download_files(products_path)
for z in project_zips:
    if z.suffix == '.nc':
        continue
    
    asfn.asf_unzip(str(products_path), str(z))
    z.unlink()

/home/jovyan/hydrosar/Dataset/Nepal_sF/RTC_GAMMA

Project: Nepal


  0%|          | 0/123 [00:00<?, ?it/s]

S1A_IW_20200323T123842_DVP_RTC30_G_gpuned_96AF.zip:   0%|          | 0/546699357 [00:00<?, ?it/s]

S1A_IW_20200416T123843_DVP_RTC30_G_gpuned_AD99.zip:   0%|          | 0/547967118 [00:00<?, ?it/s]

S1A_IW_20200404T123843_DVP_RTC30_G_gpuned_67B4.zip:   0%|          | 0/547230660 [00:00<?, ?it/s]

...

S1A_IW_20230131T123900_DVP_RTC30_G_gpuned_118D.zip:   0%|          | 0/546695642 [00:00<?, ?it/s]

S1A_IW_20221214T123902_DVP_RTC30_G_gpuned_AE57.zip:   0%|          | 0/547171908 [00:00<?, ?it/s]

S1A_IW_20230401T123900_DVP_RTC30_G_gpuned_D103.zip:   0%|          | 0/545408695 [00:00<?, ?it/s]

S1A_IW_20230730T123906_DVP_RTC30_G_gpuned_9D7F.zip:   0%|          | 0/543287387 [00:00<?, ?it/s]

S1A_IW_20230107T123901_DVP_RTC30_G_gpuned_7CF1.zip:   0%|          | 0/547130634 [00:00<?, ?it/s]

In [ ]:
rtc = batch.jobs[0].job_type == 'RTC_GAMMA'
insar = batch.jobs[0].job_type == 'INSAR_GAMMA'
autorift = batch.jobs[0].job_type == 'AUTORIFT'

**Determine the available polarizations if downloading RTC products:**

In [ ]:
if rtc:
    polarizations = asfn.get_RTC_polarizations(str(products_path))
    polarization_power_set = asfn.get_power_set(polarizations)

**Select a polarization:**

In [ ]:
if rtc:
    polarization_choice = asfn.select_parameter(sorted(polarization_power_set), 'Polarizations:')
else:
    polarization_choice = None
polarization_choice

**Create a paths variable, holding the relative path to the tiffs or NetCDFs:**

In [58]:
if rtc:
    polarization = polarization_choice.value
    print(polarization)
    if len(polarization) == 2:
        regex = "\w[\--~]{{5,300}}(_|-){}.(tif|tiff)$".format(polarization)
        dbl_polar = False
    else:
        regex = "\w[\--~]{{5,300}}(_|-){}(v|V|h|H).(tif|tiff)$".format(polarization[0])
        dbl_polar = True
elif insar:
    regex = "\w*_ueF_\w*.tif$"
elif autorift:
    # regex = "\w*ASF_OD.nc$"
    regex = "\w*ASF_OD.*$"

VH and VV


**Write functions to collect and print the paths of the tiffs or NetCDFs:**

In [59]:
def get_product_paths(regex, pths):
    product_paths = list()
    paths = Path().glob(pths)
    for pth in paths:
        tiff_path = re.search(regex, str(pth))
        if tiff_path:
            product_paths.append(pth)
    return product_paths

def print_product_paths(product_paths):
    print("Tiff paths:")
    for p in product_paths:
        print(f"{p}\n")

**Write a function to collect the product acquisition dates:**

In [60]:
def get_dates(product_paths):
    dates = []
    for pth in product_paths:
        dates.append(asfn.date_from_product_name(str(pth)).split('T')[0])
    return dates

**Convert NetCDFs to geotiffs:**

In [61]:
if autorift:
    import xarray as xr
    import re

    def ncToGeoTiff(path):   
        
        prevPath = ''
        
        for p in path.rglob('*.nc'):            
            
            layers = ['v', 'vx', 'vy', 'v_error', 'vr', 'va', 'M11', 'M12']
            fname = p.stem
            dates = list(set(re.findall(r'\d{8}', fname)))
            for layer in layers:
                layer_dir = products_path/layer

                if prevPath != p: # reduces number of runs
                    prevPath = p
                    ds = xr.open_dataset(p)
                                        
                    t1 = re.findall('\d*', ds.img_pair_info.acquisition_date_img1)
                    acq_date_1 = f'{t1[0]}T' + ''.join(t1[2:7])
                    
                    t2 = re.findall('\d*', ds.img_pair_info.acquisition_date_img2)
                    acq_date_2 = f'{t2[0]}T' + ''.join(t2[2:7])
   
                
                name = f'{fname[0:10]}_{acq_date_1}_{acq_date_2}_{fname[-6:]}_{layer}.tif'
                outfile = layer_dir/name
                                
                if not layer_dir.exists():
                    layer_dir.mkdir()
                    
                if not outfile.exists():
                    !gdal_translate NETCDF:{p}:{layer} {outfile}
            print('\n')
 
    def removeNC(path):
        for p in path.rglob('*.nc'):
            p.unlink()

**Collect and print the paths of the tiffs or NetCDFs, and collect the paths of their DEMs:**

In [62]:
rel_prod_path = products_path.relative_to(Path.cwd())
if rtc:
    product_pth = f"{str(rel_prod_path)}/*/*{polarization[0]}*.tif*"
elif insar:
    product_pth = f"{str(rel_prod_path)}/*/*.tif*"
elif autorift:
    product_pth = f"{str(rel_prod_path)}/*.tif*"    
    ncToGeoTiff(products_path)
    removeNC(products_path)

dems_pth = f"{str(rel_prod_path)}/*/*dem.tif*"

if not autorift:
    product_paths = get_product_paths(regex, product_pth)
    print_product_paths(product_paths)
else:
    print('Tiff paths:\n')
    
    for p in products_path.glob('*'):
        print(f'{p.parts[-1]}:')
        
        for p_tiff in p.rglob('*.tif'):
            print(p_tiff)
        
        print('\n')
        
# Get all paths for files ending with 'dem'
dem_paths = get_product_paths(r'dem.tif$', dems_pth)

Tiff paths:
Dataset/bang_NE/RTC_GAMMA/S1A_IW_20240118T120507_DVR_RTC30_G_gpufem_CF32/S1A_IW_20240118T120507_DVR_RTC30_G_gpufem_CF32_VV.tif

Dataset/bang_NE/RTC_GAMMA/S1A_IW_20210918T120500_DVP_RTC30_G_gpufem_6F10/S1A_IW_20210918T120500_DVP_RTC30_G_gpufem_6F10_VV.tif

Dataset/bang_NE/RTC_GAMMA/S1A_IW_20220329T120456_DVP_RTC30_G_gpufem_DA96/S1A_IW_20220329T120456_DVP_RTC30_G_gpufem_DA96_VV.tif

Dataset/bang_NE/RTC_GAMMA/S1A_IW_20201110T120454_DVP_RTC30_G_gpufem_B939/S1A_IW_20201110T120454_DVP_RTC30_G_gpufem_B939_VV.tif

Dataset/bang_NE/RTC_GAMMA/S1A_IW_20240130T120507_DVR_RTC30_G_gpufem_811D/S1A_IW_20240130T120507_DVR_RTC30_G_gpufem_811D_VV.tif

Dataset/bang_NE/RTC_GAMMA/S1A_IW_20201005T120454_DVP_RTC30_G_gpufem_37D1/S1A_IW_20201005T120454_DVP_RTC30_G_gpufem_37D1_VV.tif

Dataset/bang_NE/RTC_GAMMA/S1A_IW_20230815T120509_DVP_RTC30_G_gpufem_F226/S1A_IW_20230815T120509_DVP_RTC30_G_gpufem_F226_VV.tif

Dataset/bang_NE/RTC_GAMMA/S1A_IW_20200502T120446_DVP_RTC30_G_gpufem_D75F/S1A_IW_20200502T120

if autorift:
    - convert to geotiff
    - delete netcdfs
    - re-glob paths for geotiffs

---
## 1.2 Fix multiple UTM Zone-related issues

Fix multiple UTM Zone-related issues should they exist in your data set. If multiple UTM zones are found, the following code cells will identify the predominant UTM zone and reproject the rest into that zone. This step must be completed prior to merging frames or performing any analysis. AutoRIFT products do not come with projection metadata and so will not be reprojected.

**Use gdal.Info to determine the UTM definition types and zones in each product:**

In [63]:
if not autorift:
    coord_choice = asfn.select_parameter(["UTM", "Lat/Long"], description='Coord Systems:')
    coord_choice

In [64]:
if not autorift:
    utm_zones = []
    utm_types = []
    print('Checking UTM Zones in the data stack ...\n')
    for k in range(0, len(product_paths)):
        info = (gdal.Info(str(product_paths[k]), options = ['-json']))
        info = json.dumps(info)
        info = (json.loads(info))['coordinateSystem']['wkt']
        zone = info.split('ID')[-1].split(',')[1][0:-2]
        utm_zones.append(zone)
        typ = info.split('ID')[-1].split('"')[1]
        utm_types.append(typ)
    print(f"UTM Zones:\n {utm_zones}\n")
    print(f"UTM Types:\n {utm_types}")

Checking UTM Zones in the data stack ...

UTM Zones:
 ['32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', '32646', 

**Identify the most commonly used UTM Zone in the data:**

In [65]:
if not autorift:
    if coord_choice.value == 'UTM':
        utm_unique, counts = np.unique(utm_zones, return_counts=True)
        a = np.where(counts == np.max(counts))
        predominant_utm = utm_unique[a][0]
        print(f"Predominant UTM Zone: {predominant_utm}")
    else:
        predominant_utm = '4326'

Predominant UTM Zone: 32646


In [66]:
dem_paths

[PosixPath('Dataset/bang_NE/RTC_GAMMA/S1A_IW_20240118T120507_DVR_RTC30_G_gpufem_CF32/S1A_IW_20240118T120507_DVR_RTC30_G_gpufem_CF32_dem.tif'),
 PosixPath('Dataset/bang_NE/RTC_GAMMA/S1A_IW_20210918T120500_DVP_RTC30_G_gpufem_6F10/S1A_IW_20210918T120500_DVP_RTC30_G_gpufem_6F10_dem.tif'),
 PosixPath('Dataset/bang_NE/RTC_GAMMA/S1A_IW_20220329T120456_DVP_RTC30_G_gpufem_DA96/S1A_IW_20220329T120456_DVP_RTC30_G_gpufem_DA96_dem.tif'),
 PosixPath('Dataset/bang_NE/RTC_GAMMA/S1A_IW_20201110T120454_DVP_RTC30_G_gpufem_B939/S1A_IW_20201110T120454_DVP_RTC30_G_gpufem_B939_dem.tif'),
 PosixPath('Dataset/bang_NE/RTC_GAMMA/S1A_IW_20240130T120507_DVR_RTC30_G_gpufem_811D/S1A_IW_20240130T120507_DVR_RTC30_G_gpufem_811D_dem.tif'),
 PosixPath('Dataset/bang_NE/RTC_GAMMA/S1A_IW_20201005T120454_DVP_RTC30_G_gpufem_37D1/S1A_IW_20201005T120454_DVP_RTC30_G_gpufem_37D1_dem.tif'),
 PosixPath('Dataset/bang_NE/RTC_GAMMA/S1A_IW_20230815T120509_DVP_RTC30_G_gpufem_F226/S1A_IW_20230815T120509_DVP_RTC30_G_gpufem_F226_dem.tif'),

**Reproject all tiffs to the predominate UTM:**

In [67]:
if not autorift:
    # Reproject (if needed) and Mosaic DEM Files in Preparation for Subsequent HAND Calculation
    # print(DEM_paths)
    reproject_indicies = [i for i, j in enumerate(utm_zones) if j != predominant_utm] #makes list of indicies in utm_zones that need to be reprojected
    print('--------------------------------------------')
    print('Reprojecting %4.1f files' %(len(reproject_indicies)))
    print('--------------------------------------------')
    for k in reproject_indicies:
        temppath = f"{str(product_paths[k].parent)}/r{product_paths[k].name}"
        temppath_dem = f"{str(dem_paths[k].parent)}/r{dem_paths[k].name}"
        print(temppath)  

        cmd = f"gdalwarp -overwrite {product_paths[k]} {temppath} -s_srs {utm_types[k]}:{utm_zones[k]} -t_srs EPSG:{predominant_utm}"
    #     print(cmd)
        !{cmd}
        
        cmd = f"gdalwarp -overwrite {dem_paths[k]} {temppath_dem} -s_srs {utm_types[k]}:{utm_zones[k]} -t_srs EPSG:{predominant_utm}"
    #     print(cmd)
        !{cmd}

        product_paths[k].unlink()
        dem_paths[k].unlink()

--------------------------------------------
Reprojecting  0.0 files
--------------------------------------------


**Update product_paths with any new filenames created during reprojection:**

In [68]:
if not autorift:
    product_paths = get_product_paths(regex, product_pth)
    dem_paths = get_product_paths(r'dem.tif$', dems_pth)
    print_product_paths(product_paths)

Tiff paths:
Dataset/bang_NE/RTC_GAMMA/S1A_IW_20240118T120507_DVR_RTC30_G_gpufem_CF32/S1A_IW_20240118T120507_DVR_RTC30_G_gpufem_CF32_VV.tif

Dataset/bang_NE/RTC_GAMMA/S1A_IW_20210918T120500_DVP_RTC30_G_gpufem_6F10/S1A_IW_20210918T120500_DVP_RTC30_G_gpufem_6F10_VV.tif

Dataset/bang_NE/RTC_GAMMA/S1A_IW_20220329T120456_DVP_RTC30_G_gpufem_DA96/S1A_IW_20220329T120456_DVP_RTC30_G_gpufem_DA96_VV.tif

Dataset/bang_NE/RTC_GAMMA/S1A_IW_20201110T120454_DVP_RTC30_G_gpufem_B939/S1A_IW_20201110T120454_DVP_RTC30_G_gpufem_B939_VV.tif

Dataset/bang_NE/RTC_GAMMA/S1A_IW_20240130T120507_DVR_RTC30_G_gpufem_811D/S1A_IW_20240130T120507_DVR_RTC30_G_gpufem_811D_VV.tif

Dataset/bang_NE/RTC_GAMMA/S1A_IW_20201005T120454_DVP_RTC30_G_gpufem_37D1/S1A_IW_20201005T120454_DVP_RTC30_G_gpufem_37D1_VV.tif

Dataset/bang_NE/RTC_GAMMA/S1A_IW_20230815T120509_DVP_RTC30_G_gpufem_F226/S1A_IW_20230815T120509_DVP_RTC30_G_gpufem_F226_VV.tif

Dataset/bang_NE/RTC_GAMMA/S1A_IW_20200502T120446_DVP_RTC30_G_gpufem_D75F/S1A_IW_20200502T120

---
## 1.3 Merge multiple frames from the same date.

You may notice duplicates in your acquisition dates. As HyP3 processes SAR data on a frame-by-frame basis, duplicates may occur if your area of interest is covered by two consecutive image frames. In this case, two separate images are generated that need to be merged together before time series processing can commence. Currently we only merge RTCs.

**Create a directory in which to store the reprojected and merged RTCs, and one directory for the DEMs:**

In [69]:
while True:
    print(f"Current working directory: {analysis_directory}")
    name = (input(f"\nPlease enter the name of a directory in which to store the tiffs and dems."))
    output_dir_path_tiffs = analysis_directory.joinpath(f"{job_type.value}_{name}_tiffs")
    output_dir_path_dem = analysis_directory.joinpath(f"{job_type.value}_{name}_dems")
    
    if output_dir_path_tiffs == Path('.'):
        continue
    if output_dir_path_tiffs.is_dir():
        contents = output_dir_path_tiffs.glob('*')
        if len(list(contents)) > 0:
            choice = asfn.handle_old_data(output_dir_path_tiffs)
            if choice == 1:
                if output_dir_path_tiffs.exists():
                    shutil.rmtree(output_dir_path_tiffs)
                output_dir_path_tiffs.mkdir(parents=True, exist_ok=True)
                output_dir_path_dem.mkdir(parents=True, exist_ok=True)
                break
            elif choice == 2:
                break
            else:
                clear_output()
                continue
        else:
            break
    else:
        output_dir_path_tiffs.mkdir(parents=True, exist_ok=True)
        output_dir_path_dem.mkdir(parents=True, exist_ok=True)
        break

Current working directory: /home/jovyan/hydrosar/Dataset/bang_NE



Please enter the name of a directory in which to store the tiffs and dems. 



********************** WARNING! **********************
The directory /home/jovyan/hydrosar/Dataset/bang_NE/RTC_GAMMA__tiffs already exists and contains:
• S1A_IW_20190719T120444_DVP_RTC30_G_gpufem_DAFE_VH.tif
• S1A_IW_20220422T120457_DVP_RTC30_G_gpufem_D66B_VH.tif
• S1A_IW_20211105T120500_DVP_RTC30_G_gpufem_FA5C_VH.tif
• S1A_IW_20190905T120447_DVP_RTC30_G_gpufem_AF79_VH.tif
• S1A_IW_20200103T120446_DVP_RTC30_G_gpufem_DBED_VH.tif
• S1A_IW_20231225T120509_DVP_RTC30_G_gpufem_C6D6_VH.tif
• S1A_IW_20200127T120445_DVP_RTC30_G_gpufem_5E3A_VH.tif
• S1A_IW_20210214T120450_DVP_RTC30_G_gpufem_2A53_VH.tif
• S1A_IW_20211211T120459_DVP_RTC30_G_gpufem_3B81_VH.tif
• S1A_IW_20210720T120457_DVP_RTC30_G_gpufem_A86B_VH.tif
• S1A_IW_20190601T120441_DVP_RTC30_G_gpufem_6710_VH.tif
• S1A_IW_20210708T120456_DVP_RTC30_G_gpufem_9048_VH.tif
• S1A_IW_20191128T120447_DVP_RTC30_G_gpufem_F1A1_VH.tif
• S1A_IW_20191104T120448_DVP_RTC30_G_gpufem_247C_VH.tif
• S1A_IW_20210322T120450_DVP_RTC30_G_gpufem_9E55_VH.tif
• S1A_

Select option 1, 2, or 3.
 2


**Create a set from the date list, removing any duplicates:**

In [70]:
if rtc:
    dates = get_dates(product_paths)
    print(dates)
    unique_dates = set(dates)
    print(unique_dates)

['20240118', '20210918', '20220329', '20201110', '20240130', '20201005', '20230815', '20200502', '20200725', '20201029', '20210720', '20230616', '20210930', '20231026', '20220305', '20210521', '20210813', '20220221', '20230523', '20201122', '20190201', '20191023', '20190601', '20200420', '20210801', '20231213', '20200514', '20211012', '20191128', '20190426', '20211024', '20200127', '20201228', '20200103', '20211105', '20231119', '20210403', '20221218', '20190905', '20210415', '20210906', '20220621', '20211117', '20230312', '20200713', '20230628', '20230827', '20190812', '20190520', '20210427', '20221031', '20230204', '20230324', '20230803', '20231225', '20221124', '20200923', '20190309', '20231201', '20210109', '20230920', '20210825', '20190929', '20211129', '20220128', '20220209', '20230604', '20190731', '20200408', '20211211', '20190719', '20230417', '20200607', '20220422', '20201204', '20190414', '20211223', '20201017', '20230722', '20190321', '20200619', '20210322', '20200208', '20

**Determine which dates have multiple frames. Create a dictionary with each date as a key linked to a value set as an empty string:**

In [71]:
if rtc: 
    dup_date_batches = [{}]
    for date in unique_dates:
        count = 0
        for d in dates:
            if date == d:
                count +=1
        if (dbl_polar and count > 2) or (not dbl_polar and count > 1):
            dup_date_batches[0].update({date : ""})
    if dbl_polar:
        dup_date_batches.append(copy.deepcopy(dup_date_batches[0]))
    print(dup_date_batches)

[{}, {}]


**Update the key values in dup_paths with the string paths to all the tiffs for each date:**

In [72]:
if rtc:
    if dbl_polar:
        polar_list = [polarization.split(' ')[0], polarization.split(' ')[2]]
    else:
        polar_list = [polarization]

    for i, polar in enumerate(polar_list):
        polar_path_regex = f"(\w|/)*_{polar}.(tif|tiff)$"
        polar_paths = get_product_paths(polar_path_regex, product_pth)
        for pth in polar_paths:
            date = asfn.date_from_product_name(str(pth)).split('T')[0]
            if date in dup_date_batches[i]:
                dup_date_batches[i][date] = f"{dup_date_batches[i][date]} {str(pth)}"

    for d in dup_date_batches:
        print(d)
        print("\n")

{}


{}




**Merge all the frames for each date, save the results to the output directory, and delete the original tiffs.**

In [73]:
if rtc and len(dup_date_batches[0]) > 0:
    for i, dup_dates in enumerate(dup_date_batches):
        polar_regex = "(?<=_)(vh|VH|vv|VV)(?=.tif{1,2})"
        polar = re.search(polar_regex, dup_dates[list(dup_dates)[0]])
        if polar:
            polar = f'_{polar.group(0)}'
        else:
            polar = ''
        for dup_date in dup_dates:
#             print(f"\n\n{dup_dates[dup_date]}")
            output = f"{str(output_dir_path_tiffs)}/merged_{dup_date}T999999{polar}{product_paths[0].suffix}"
            gdal_command = f"gdal_merge.py -o {output} {dup_dates[dup_date]}"
            print(f"\n\nCalling the command: {gdal_command}\n")
            !$gdal_command
            for pth in dup_dates[dup_date].split(' '):
                path = Path(pth)
                if path and path.is_file():
                    path.unlink()
                    print(f"Deleting: {str(pth)}")

---
**Verify that all duplicate dates were resolved:**

In [74]:
if rtc:
    product_paths = get_product_paths(regex, product_pth)
    for polar in polar_list:
        polar_product_pth = product_pth.replace('V*', polar)
        polar_product_paths = get_product_paths(regex, polar_product_pth)
        dates = get_dates(polar_product_paths)
        if len(dates) != len(set(dates)):
            print(f"Duplicate dates still present!")
        else:
            print(f"No duplicate dates are associated with {polar} polarization.")

No duplicate dates are associated with VH polarization.
No duplicate dates are associated with VV polarization.


**Print the updated the paths to all remaining non-merged tiffs:**

In [75]:
if rtc:
    print_product_paths(product_paths)

Tiff paths:
Dataset/bang_NE/RTC_GAMMA/S1A_IW_20240118T120507_DVR_RTC30_G_gpufem_CF32/S1A_IW_20240118T120507_DVR_RTC30_G_gpufem_CF32_VV.tif

Dataset/bang_NE/RTC_GAMMA/S1A_IW_20210918T120500_DVP_RTC30_G_gpufem_6F10/S1A_IW_20210918T120500_DVP_RTC30_G_gpufem_6F10_VV.tif

Dataset/bang_NE/RTC_GAMMA/S1A_IW_20220329T120456_DVP_RTC30_G_gpufem_DA96/S1A_IW_20220329T120456_DVP_RTC30_G_gpufem_DA96_VV.tif

Dataset/bang_NE/RTC_GAMMA/S1A_IW_20201110T120454_DVP_RTC30_G_gpufem_B939/S1A_IW_20201110T120454_DVP_RTC30_G_gpufem_B939_VV.tif

Dataset/bang_NE/RTC_GAMMA/S1A_IW_20240130T120507_DVR_RTC30_G_gpufem_811D/S1A_IW_20240130T120507_DVR_RTC30_G_gpufem_811D_VV.tif

Dataset/bang_NE/RTC_GAMMA/S1A_IW_20201005T120454_DVP_RTC30_G_gpufem_37D1/S1A_IW_20201005T120454_DVP_RTC30_G_gpufem_37D1_VV.tif

Dataset/bang_NE/RTC_GAMMA/S1A_IW_20230815T120509_DVP_RTC30_G_gpufem_F226/S1A_IW_20230815T120509_DVP_RTC30_G_gpufem_F226_VV.tif

Dataset/bang_NE/RTC_GAMMA/S1A_IW_20200502T120446_DVP_RTC30_G_gpufem_D75F/S1A_IW_20200502T120

**Merge all the DEMs into one (overlapping data should be the same because they are all from COP30)**

In [76]:
# List of DEM file paths
dem_paths

# List to store the opened DEM datasets
datasets = []

# Open each DEM dataset and add it to the list
for file in dem_paths:
    dataset = rasterio.open(file)
    datasets.append(dataset)

# Merge the DEM datasets
merged, out_transform = merge(datasets)

**Move all remaining unmerged tiffs into the output directory, and choose whether to save or delete the directory holding the remaining downloaded product files. AutoRIFT NetCDFs will remain in their original directory:**

In [77]:
if not autorift:
    choices = ['save', 'delete']
    print("Do you wish to save or delete the directory containing auxiliary product files?")
else:
    choices = []
save_or_del = asfn.select_parameter(choices)
save_or_del

Do you wish to save or delete the directory containing auxiliary product files?


RadioButtons(layout=Layout(min_width='800px'), options=('save', 'delete'), value='save')

In [78]:
product_paths


[PosixPath('Dataset/bang_NE/RTC_GAMMA/S1A_IW_20240118T120507_DVR_RTC30_G_gpufem_CF32/S1A_IW_20240118T120507_DVR_RTC30_G_gpufem_CF32_VV.tif'),
 PosixPath('Dataset/bang_NE/RTC_GAMMA/S1A_IW_20210918T120500_DVP_RTC30_G_gpufem_6F10/S1A_IW_20210918T120500_DVP_RTC30_G_gpufem_6F10_VV.tif'),
 PosixPath('Dataset/bang_NE/RTC_GAMMA/S1A_IW_20220329T120456_DVP_RTC30_G_gpufem_DA96/S1A_IW_20220329T120456_DVP_RTC30_G_gpufem_DA96_VV.tif'),
 PosixPath('Dataset/bang_NE/RTC_GAMMA/S1A_IW_20201110T120454_DVP_RTC30_G_gpufem_B939/S1A_IW_20201110T120454_DVP_RTC30_G_gpufem_B939_VV.tif'),
 PosixPath('Dataset/bang_NE/RTC_GAMMA/S1A_IW_20240130T120507_DVR_RTC30_G_gpufem_811D/S1A_IW_20240130T120507_DVR_RTC30_G_gpufem_811D_VV.tif'),
 PosixPath('Dataset/bang_NE/RTC_GAMMA/S1A_IW_20201005T120454_DVP_RTC30_G_gpufem_37D1/S1A_IW_20201005T120454_DVP_RTC30_G_gpufem_37D1_VV.tif'),
 PosixPath('Dataset/bang_NE/RTC_GAMMA/S1A_IW_20230815T120509_DVP_RTC30_G_gpufem_F226/S1A_IW_20230815T120509_DVP_RTC30_G_gpufem_F226_VV.tif'),
 Posix

In [79]:
if not autorift:
    for tiff in product_paths:
        tiff.rename(f"{output_dir_path_tiffs}/{tiff.name}")
    
    # Save the merged DEM
    # Define the output file path and metadata
    output_file = f"{output_dir_path_dem}/{name}_merged.tif"
    output_meta = dataset.meta.copy()
    output_meta.update({'driver': 'GTiff', 'height': merged.shape[1], 'width': merged.shape[2], 'transform': out_transform})
    # Write the merged DEM to the output file
    with rasterio.open(output_file, 'w', **output_meta) as dst:
        dst.write(merged)
    # Close all the opened datasets
    for dataset in datasets:
        dataset.close()

    if save_or_del.value == 'delete':
        shutil.rmtree(products_path)
    product_paths = get_product_paths(regex, product_pth)

**Print the path where you saved your tiffs or NetCDFs.**

In [80]:
if rtc or insar:
    print(str(output_dir_path_tiffs))
elif autorift:
    print(str(products_path))

/home/jovyan/hydrosar/Dataset/bang_NE/RTC_GAMMA__tiffs


*Prepare_RTC_Stack_HyP3_v2.ipynb - Version 2.0.2 - June 2023*

*Version Changes:*
- *added dems migration to a folder to be used in HAND*
- *fix breaking projectless jobs downloading*